In [18]:
df_final = pd.read_csv("df_final_creuse.csv")


In [19]:
import pandas as pd
import glob
import os
import csv

def load_split_csv(file_prefix, folder='data'):
    
    search_pattern = os.path.join(folder, f"{file_prefix}*")
    files = sorted(glob.glob(search_pattern))
    
    if not files:
        print(search_pattern)
        return pd.DataFrame()
    
    
    df_list = []
    for f in files:
        try:
            # quoting=csv.QUOTE_NONE 
           
            df = pd.read_csv(
                f, 
                engine='python', 
                quoting=csv.QUOTE_NONE, 
                on_bad_lines='skip', 
                encoding='utf-8'
            )
            df_list.append(df)
        except Exception as e:
            print(f"Error reading {f}: {e}")
        

           
    
    if not df_list:
        return pd.DataFrame()
        
    return pd.concat(df_list, ignore_index=True)

df_movie = load_split_csv('movie_part_', folder='../data')
df_people = load_split_csv('people_temp_part_', folder='../data')
df_tmdb = load_split_csv('tmdb_full_part_', folder='.')

print(f"Movie rows: {len(df_movie)}")
print(f"People rows: {len(df_people)}")
print(f"TMDB rows: {len(df_tmdb)}")

Movie rows: 79088
People rows: 2981595
TMDB rows: 64715


In [39]:
df = df_final.copy()

# Filtre Art & Essai / période pertinente
df = df[df['year'] >= 1960]

# Sécurité
df['overview'] = df['overview'].fillna('')
df['poster_path'] = df['poster_path'].fillna('')
df['actors'] = df['actors'].fillna('')
df['producers'] = df['producers'].fillna('')
df['genres'] = df['genres'].fillna('')



In [40]:

df = pd.read_csv('../data/movie.csv')
df.shape


(177495, 12)

In [21]:
#recommandation par GENRES
#Préparer les GENRES pour la recommandation
df['genres'].head()


0                        ['Comedy']
1              ['Drama', 'Romance']
2      ['Drama', 'Comedy', 'Crime']
3    ['Drama', 'Comedy', 'Romance']
4               ['Crime', 'Comedy']
Name: genres, dtype: object

In [22]:
# transformer ce texte en vraie liste
#mettre tout en minuscule
#préparer une version “texte” pour calculer la similarité
import ast

df['genres_list'] = df['genres'].apply(ast.literal_eval)  #convertir le texte en liste
df['genres_list'].head()


0                    [Comedy]
1            [Drama, Romance]
2      [Drama, Comedy, Crime]
3    [Drama, Comedy, Romance]
4             [Crime, Comedy]
Name: genres_list, dtype: object

In [44]:
import ast

df['genres_list'] = df['genres'].apply(ast.literal_eval)
df['genres_list'] = df['genres_list'].apply(
    lambda x: [g.lower() for g in x]
)

df['genres_text'] = df['genres_list'].apply(lambda x: ' '.join(x))
df[['title', 'genres_text']].head()



,title,genres_text
0,Blondie,comedy
1,Love at Twenty,drama romance
2,Ariel,drama comedy crime
3,Shadows in Paradise,drama comedy romance
4,Four Rooms,crime comedy


In [42]:
df['genres_text'] = df['genres_list'].apply(
    lambda x: ' '.join([g.lower() for g in x])
)

df[['title', 'genres_text']].head()




,title,genres_text
0,Blondie,comedy
1,Love at Twenty,drama romance
2,Ariel,drama comedy crime
3,Shadows in Paradise,drama comedy romance
4,Four Rooms,crime comedy


In [45]:
#préparer une version “texte” pour calculer la similarité
df['genres_text'] = df['genres_list'].apply(lambda x: ' '.join(x))
df[['title', 'genres_text']].head()


,title,genres_text
0,Blondie,comedy
1,Love at Twenty,drama romance
2,Ariel,drama comedy crime
3,Shadows in Paradise,drama comedy romance
4,Four Rooms,crime comedy


In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['genres_text'])

tfidf_matrix.shape


(177495, 21)

In [47]:
from sklearn.neighbors import NearestNeighbors #iKNN non supervise, importer le modèle de k plus proches voisins

knn = NearestNeighbors(
    n_neighbors=6,   # 1 = le film lui-même + 5 recommandations
    metric='euclidean'
)

knn.fit(tfidf_matrix)


,"n_neighbors n_neighbors: int, default=5Number of neighbors to use by default for :meth:`kneighbors` queries.",6
,"radius radius: float, default=1.0Range of parameter space to use by default for :meth:`radius_neighbors`queries.",1.0
,"algorithm algorithm: {'auto', 'ball_tree', 'kd_tree', 'brute'}, default='auto'Algorithm used to compute the nearest neighbors:- 'ball_tree' will use :class:`BallTree`- 'kd_tree' will use :class:`KDTree`- 'brute' will use a brute-force search.- 'auto' will attempt to decide the most appropriate algorithm based on the values passed to :meth:`fit` method.Note: fitting on sparse input will override the setting ofthis parameter, using brute force.",'auto'
,"leaf_size leaf_size: int, default=30Leaf size passed to BallTree or KDTree. This can affect thespeed of the construction and query, as well as the memoryrequired to store the tree. The optimal value depends on thenature of the problem.",30
,"metric metric: str or callable, default='minkowski'Metric to use for distance computation. Default is ""minkowski"", whichresults in the standard Euclidean distance when p = 2. See thedocumentation of `scipy.spatial.distance`_ andthe metrics listed in:class:`~sklearn.metrics.pairwise.distance_metrics` for valid metricvalues.If metric is ""precomputed"", X is assumed to be a distance matrix andmust be square during fit. X may be a :term:`sparse graph`, in whichcase only ""nonzero"" elements may be considered neighbors.If metric is a callable function, it takes two arrays representing 1Dvectors as inputs and must return one value indicating the distancebetween those vectors. This works for Scipy's metrics, but is lessefficient than passing the metric name as a string.",'euclidean'
,"p p: float (positive), default=2Parameter for the Minkowski metric fromsklearn.metrics.pairwise.pairwise_distances. When p = 1, this isequivalent to using manhattan_distance (l1), and euclidean_distance(l2) for p = 2. For arbitrary p, minkowski_distance (l_p) is used.",2
,"metric_params metric_params: dict, default=NoneAdditional keyword arguments for the metric function.",None
,"n_jobs n_jobs: int, default=NoneThe number of parallel jobs to run for neighbors search.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None


In [49]:
def recommend_by_genres(title, df=df, model=knn, n_reco=5):
    # verifier que le film existe
    if title not in df['title'].values:
        return "Film non trouvé"

    # recupiration  l'index du film
    idx = df[df['title'] == title].index[0]

    #hercher les voisins avec KNN
    distances, indices = model.kneighbors(
        tfidf_matrix[idx],
        n_neighbors=n_reco + 1
    )

    # enlever le film lui-même
    similar_indices = indices[0][1:]

    # retourner les films recommandés
    return df.loc[similar_indices, ['title', 'genres_text', 'rating']]


In [50]:
#Tester la fonction
recommend_by_genres("Blondie")


,title,genres_text,rating
49958,Nick Offerman & Megan Mullally - Summer of 69:...,comedy,5.000
49957,Myths,comedy,3.400
66454,How to Plan an Orgy in a Small Town,comedy,5.186
66452,Why Him?,comedy,6.372
66492,The Target,comedy,5.500


In [51]:
# Vérifier les titres en double
df['title'].value_counts().head()


title
Cinderella           28
A Christmas Carol    23
Alone                21
Home                 20
Mother               18
Name: count, dtype: int64

In [52]:
df = df_final.copy()

df = df.drop_duplicates(subset='imdb_id').reset_index(drop=True)


In [53]:


df_final = pd.read_csv("df_final_creuse.csv", low_memory=False)


In [54]:
df.info()
df.head(3)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177495 entries, 0 to 177494
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   imdb_id      177495 non-null  object 
 1   title        177495 non-null  object 
 2   genres       177495 non-null  object 
 3   overview     171888 non-null  object 
 4   rating       177495 non-null  float64
 5   runtime      177495 non-null  int64  
 6   poster_path  168060 non-null  object 
 7   popularity   177495 non-null  float64
 8   year         177359 non-null  float64
 9   numVotes     177495 non-null  float64
 10  actors       159130 non-null  object 
 11  producers    145623 non-null  object 
dtypes: float64(4), int64(1), object(7)
memory usage: 16.3+ MB


,imdb_id,title,genres,overview,rating,runtime,poster_path,popularity,year,numVotes,actors,producers
0,tt0029927,Blondie,['Comedy'],Blondie and Dagwood are about to celebrate the...,7.214,70,/zBiHKhXklvTFwj4M1uEUcQGAVJ.jpg,2.852,1938.0,927.0,"Penny Singleton, Arthur Lake, Larry Simms, Dai...",NaN
1,tt0055747,Love at Twenty,"['Drama', 'Romance']",Love at Twenty unites five directors from five...,6.700,110,/aup2QCYCsyEeQfpboXy0f4uj8aE.jpg,3.770,1962.0,2558.0,"Jean-Pierre Léaud, Marie-France Pisier, Eleono...",Pierre Roustang
2,tt0094675,Ariel,"['Drama', 'Comedy', 'Crime']",Taisto Kasurinen is a Finnish coal miner whose...,7.046,73,/ojDg0PGvs6R9xYFodRct2kdI6wC.jpg,9.214,1988.0,9586.0,"Turo Pajala, Susanna Haavisto, Matti Pellonpää...",Aki Kaurismäki


In [ ]:
def recommend_by_genres(title, df=df, model=knn, n_reco=5):
